# 5.7 로지스틱 회귀 - 분류

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

In [4]:
cancer = load_breast_cancer()

In [8]:
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [9]:
from sklearn.preprocessing import StandardScaler # 데이터 정규화
from sklearn.model_selection import train_test_split 

In [10]:
# StandardScaler( )로 평균이 0, 분산 1로 데이터 분포도 변환
# cancer.data : 학습, 테스트할 데이터 정규화
scaler = StandardScaler()
data_scaled = scaler.fit_transform(cancer.data)

In [11]:
X_train , X_test, y_train , y_test =\
train_test_split(data_scaled, cancer.target, test_size=0.3, random_state=0)

- 데이터 준비 완료

### 평가

In [13]:
# accuracy_score : 정확도 점수
# roc_auc_score : 정밀도/재현율 점수
from sklearn.metrics import accuracy_score, roc_auc_score

In [15]:
# 로지스틱 회귀를 이용하여 학습 및 예측 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train) # 모델(식) 생성
lr_preds = lr_clf.predict(X_test) # 테스트 데이터로 예측값 추출

In [16]:
# accuracy와 roc_auc 측정
print('accuracy: {:0.3f}'.format(accuracy_score(y_test, lr_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_test , lr_preds)))

accuracy: 0.977
roc_auc: 0.972


- 혼돈행렬을 뽑아낼 필요가 있음
- 정밀도, 재현율 점수를 뽑아낼 필요가 있음

#### Regression => 규제를 적용할 수 있음 : L1(절대값 처리), L2 (가중치에 제곱)
- L1 (절대값 처리) : 중요도 적은 컬럼의 weight를 0
- L2 (제곱) : 중요도 적은 컬럼의 Weight를 작게 만듦

#### LogisticRegression => parameter : panalty라는 파라미터가 존재
- panalty('L1,L2')

- L1, L2 => alpha 값을 줘야 함 (alpha 값도 파라미터에 넣어야 함) => C parameter에 적용

### LogisticRegression에 panalty라는 parameter와 C라는 parameter가 존재

In [18]:
# Regression => 규제 적용 가능 : l1(절대값처리), l2(제곱)
# l1(절대값처리) : 중요도 적은 컬럼의 Weight를 0 
# l2(제곱) : 중요도 적은 컬럼의 Weight를 작게 만든다
# Logistic Regression => parameter : penalty('l1, l2') 
# l1, l2 => alpha 파라미터 => C paramter = 1 / alpha
from sklearn.model_selection import GridSearchCV
params={'penalty':['l2', 'l1'], # 2개
        'C':[0.01, 0.1, 1, 5, 10]} #5개

### GridSearchCV를 돌려봄

In [19]:
grid_clf = GridSearchCV(
      lr_clf
    , param_grid=params
    , scoring='accuracy'
    , cv=3 
)
grid_clf.fit(data_scaled, cancer.target)

/home/dpqls03/anaconda3/envs/beef-stock/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/dpqls03/anaconda3/envs/beef-stock/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dpqls03/anaconda3/envs/beef-stock/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/dpqls03/anaconda3/envs/beef-stock/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/home/dpqls03/anaconda3/envs/beef-stock/lib/pytho

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 5, 10], 'penalty': ['l2', 'l1']},
             scoring='accuracy')

In [20]:
best_str = '최적 하이퍼 파라미터:{0}, 최적 평균 정확도:{1:.3f}' # 소수점 세번째까지
print(best_str.format(
    grid_clf.best_params_ # 최적 파라미터
    , grid_clf.best_score_ # 최적 평균 정확도 점수
))

최적 하이퍼 파라미터:{'C': 1, 'penalty': 'l2'}, 최적 평균 정확도:0.975


#### LogisticRegression의 parameter
1. penalty
2. C